In [47]:
from IPython.display import clear_output
import sys
import numpy as np
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from util.algorithms import run_sarsa

In [48]:
# Verifica se está rodando no Google Colab
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !git clone https://github.com/LucaLemos/UFRPE_AprendizagemReforco
    sys.path.append("/content/UFRPE_AprendizagemReforco")
    clear_output()
    !apt-get install ffmpeg
    !pip install gymnasium==1.0.0
else:
    from os import path
    sys.path.append(path.dirname(path.dirname(path.abspath("__main__"))))

In [49]:
# Hiperparâmetros Globais
GAMMA = 0.95  # Fator de desconto
TAU = 0.005   # Taxa de atualização da target_network
ALPHA = 0.5   # Peso do termo CQL
ITERATIONS = 5000  # Número de iterações do CQL
DATASET_SIZE = 1_000_000  # Tamanho do conjunto de dados (replay buffer)
BATCH_SIZE = 128  # Tamanho do batch para treinamento da rede neural
LEARNING_RATE = 1e-3  # Taxa de aprendizado para o otimizador
TARGET_UPDATE_FREQ = 200  # Atualizar a target network a cada 200 iterações

In [50]:
# Definição da Rede Q
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, action_dim)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [51]:
# Função para converter estados em tensores one-hot
def to_one_hot(state, num_states):
    one_hot = np.zeros(num_states)
    one_hot[state] = 1
    return torch.FloatTensor(one_hot)

In [52]:
# Função para amostrar um batch do replay buffer
def sample_batch(replay_buffer, batch_size, states_dim):
    """Retorna um batch aleatório de transições do buffer."""
    indices = np.random.choice(len(replay_buffer), batch_size, replace=False)
    batch = [replay_buffer[i] for i in indices]

    # Extrai apenas os 5 elementos necessários
    states, actions, rewards, next_states, dones = zip(*[(s, a, r, ns, d) for (s, a, r, ns, _, _, d, _) in batch])

    # Converte para tensores
    states = torch.stack([to_one_hot(s, states_dim) for s in states])
    next_states = torch.stack([to_one_hot(ns, states_dim) for ns in next_states])
    actions = torch.LongTensor(actions).unsqueeze(1)
    rewards = torch.FloatTensor(rewards).unsqueeze(1)
    dones = torch.FloatTensor(dones).unsqueeze(1)

    return states, actions, rewards, next_states, dones

In [53]:
# Função para calcular a perda CQL
def cql_loss(q_network, states, actions, target_q_values, alpha=0.5):
    q_values = q_network(states).gather(1, actions)
    all_q_values = q_network(states)
    logsumexp_q = torch.logsumexp(all_q_values, dim=1, keepdim=True)
    penalty = logsumexp_q - q_values
    loss = torch.mean((q_values - target_q_values) ** 2) + alpha * torch.mean(penalty)
    return loss

In [54]:
# Função para treinar o modelo CQL
def train_cql(q_network, target_network, replay_buffer, states_dim, q_values_means, rewards_variances, batch_size=64, optimizer=None, alpha=0.5, gamma=0.95):
    states, actions, rewards, next_states, dones = sample_batch(replay_buffer, batch_size, states_dim)
    q_values = q_network(states).gather(1, actions)
    with torch.no_grad():
        next_q_values = target_network(next_states).max(1, keepdim=True)[0]
        target_q_values = rewards + gamma * next_q_values * (1 - dones)
    loss = cql_loss(q_network, states, actions, target_q_values, alpha)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    q_values_means.append(q_values.mean().item())
    rewards_variances.append(torch.var(rewards).item())
    return loss, q_values_means, rewards_variances

In [55]:
# Função para avaliar a política
def evaluate_policy(q_network, env, states_dim, num_episodes=100, max_steps=50000):
    total_rewards = []
    for _ in range(num_episodes):
        state, _ = env.reset()
        done = False
        total_reward = 0
        step = 0
        while not done and step < max_steps:
            state_one_hot = to_one_hot(state, states_dim).unsqueeze(0)
            q_values = q_network(state_one_hot)
            action = torch.argmax(q_values).item()
            state, reward, done, _, _ = env.step(action)
            total_reward += reward
            step += 1
        total_rewards.append(total_reward)
    return np.mean(total_rewards)

In [56]:
# Função para atualizar a target network
def update_target_network(q_network, target_network, tau=0.005):
    for target_param, param in zip(target_network.parameters(), q_network.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

In [57]:
# Função para treinar e salvar o modelo
def train_and_save_model(env_name, env, replay_buffer, states_dim, actions_dim, batch_size, learning_rate, gamma, alpha, iterations, evaluate_every, target_update_freq, model_save_path):
    q_network = QNetwork(states_dim, actions_dim)
    target_network = QNetwork(states_dim, actions_dim)
    target_network.load_state_dict(q_network.state_dict())
    optimizer = optim.Adam(q_network.parameters(), lr=learning_rate)
    losses = []
    avg_rewards = []
    q_values_means = []
    rewards_variances = []
    for iteration in range(iterations):
        loss, q_values_means, rewards_variances = train_cql(q_network, target_network, replay_buffer, states_dim, q_values_means, rewards_variances, batch_size, optimizer, alpha, gamma)
        if (iteration + 1) % target_update_freq == 0:
            update_target_network(q_network, target_network, TAU)
        if (iteration + 1) % evaluate_every == 0:
            avg_reward = evaluate_policy(q_network, env, states_dim)
            avg_rewards.append(avg_reward)
            losses.append(loss.item())
            print(f"Iteração {iteration + 1}, Perda: {loss.item()}, Recompensa Média: {avg_reward}")
    torch.save(q_network.state_dict(), model_save_path)
    print(f"Modelo salvo em {model_save_path}")
    return q_network, losses, avg_rewards

# FrozenLake

In [58]:
# Função específica para FrozenLake
def train_frozenlake():
    env_name = "FrozenLake-v1"
    env = gym.make(env_name, render_mode="rgb_array")
    states_dim = env.observation_space.n
    actions_dim = env.action_space.n
    model_save_path = f"model/q_network_{env_name}.pth"

    # Hiperparâmetros específicos para FrozenLake
    batch_size = 128
    learning_rate = 1e-3
    gamma = 0.95
    alpha = 0.5
    iterations = 5000
    evaluate_every = 1000
    target_update_freq = 200

    # Coleta dados
    sum_rewards_per_ep, _, transitions = run_sarsa(env, DATASET_SIZE, learning_rate, gamma)
    replay_buffer = transitions

    # Treina e salva o modelo
    q_network, losses, avg_rewards = train_and_save_model(
        env_name, env, replay_buffer, states_dim, actions_dim, batch_size, learning_rate,
        gamma, alpha, iterations, evaluate_every, target_update_freq, model_save_path
    )
    print(f"Ambiente: {env_name}")
    print(f"Recompensa média final: {np.mean(avg_rewards)}")
    print(f"Variância da recompensa: {np.var(avg_rewards)}")
    print("-" * 50)

In [59]:
train_frozenlake()

Run sarsa in <TimeLimit<OrderEnforcing<PassiveEnvChecker<FrozenLakeEnv<FrozenLake-v1>>>>>
Episódio 0 terminou com recompensa 0.0 na transição 29
Episódio 100 terminou com recompensa 0.0 na transição 1517
Episódio 200 terminou com recompensa 0.0 na transição 3253
Episódio 300 terminou com recompensa 0.0 na transição 4897
Episódio 400 terminou com recompensa 0.0 na transição 6447
Episódio 500 terminou com recompensa 0.0 na transição 8067
Episódio 600 terminou com recompensa 0.0 na transição 9669
Episódio 700 terminou com recompensa 0.0 na transição 11228
Episódio 800 terminou com recompensa 0.0 na transição 12742
Episódio 900 terminou com recompensa 0.0 na transição 14290
Episódio 1000 terminou com recompensa 0.0 na transição 15726
Episódio 1100 terminou com recompensa 0.0 na transição 16911
Episódio 1200 terminou com recompensa 1.0 na transição 18235
Episódio 1300 terminou com recompensa 0.0 na transição 19509
Episódio 1400 terminou com recompensa 0.0 na transição 20746
Episódio 1500 te

# Taxi

In [60]:
# Função específica para Taxi
def train_taxi():
    env_name = "Taxi-v3"
    env = gym.make(env_name, render_mode="rgb_array")
    states_dim = env.observation_space.n
    actions_dim = env.action_space.n
    model_save_path = f"model/q_network_{env_name}.pth"

    # Hiperparâmetros específicos para Taxi
    batch_size = 256
    learning_rate = 1e-4  # Taxa de aprendizado menor
    gamma = 0.99  # Fator de desconto maior
    alpha = 0.2
    iterations = 5000  # Número maior de iterações
    evaluate_every = 1000
    target_update_freq = 500

    # Coleta dados
    sum_rewards_per_ep, _, transitions = run_sarsa(env, DATASET_SIZE, learning_rate, gamma)
    replay_buffer = transitions

    # Treina e salva o modelo
    q_network, losses, avg_rewards = train_and_save_model(
        env_name, env, replay_buffer, states_dim, actions_dim, batch_size, learning_rate,
        gamma, alpha, iterations, evaluate_every, target_update_freq, model_save_path
    )
    print(f"Ambiente: {env_name}")
    print(f"Recompensa média final: {np.mean(avg_rewards)}")
    print(f"Variância da recompensa: {np.var(avg_rewards)}")
    print("-" * 50)

In [61]:
train_taxi()

Run sarsa in <TimeLimit<OrderEnforcing<PassiveEnvChecker<TaxiEnv<Taxi-v3>>>>>
Episódio 0 terminou com recompensa -686 na transição 199
Episódio 100 terminou com recompensa -254 na transição 18943
Episódio 200 terminou com recompensa -317 na transição 37871
Episódio 300 terminou com recompensa -299 na transição 56789
Episódio 400 terminou com recompensa -245 na transição 75934
Episódio 500 terminou com recompensa -254 na transição 94625
Episódio 600 terminou com recompensa -254 na transição 113681
Episódio 700 terminou com recompensa -272 na transição 133049
Episódio 800 terminou com recompensa -41 na transição 152134
Episódio 900 terminou com recompensa -308 na transição 171298
Episódio 1000 terminou com recompensa -389 na transição 190587
Episódio 1100 terminou com recompensa -272 na transição 209553
Episódio 1200 terminou com recompensa -245 na transição 228628
Episódio 1300 terminou com recompensa -263 na transição 247836
Episódio 1400 terminou com recompensa -204 na transição 26670

# Cliff

In [62]:
# Função específica para CliffWalking
def train_cliffwalking():
    env_name = "CliffWalking-v0"
    env = gym.make(env_name, render_mode="rgb_array")
    states_dim = env.observation_space.n
    actions_dim = env.action_space.n
    model_save_path = f"model/q_network_{env_name}.pth"

    # Hiperparâmetros específicos para CliffWalking
    batch_size = 64
    learning_rate = 1e-3
    gamma = 0.9
    alpha = 0.1
    iterations = 3000
    evaluate_every = 500
    target_update_freq = 100

    # Coleta dados
    sum_rewards_per_ep, _, transitions = run_sarsa(env, DATASET_SIZE, learning_rate, gamma)
    replay_buffer = transitions

    # Treina e salva o modelo
    q_network, losses, avg_rewards = train_and_save_model(
        env_name, env, replay_buffer, states_dim, actions_dim, batch_size, learning_rate,
        gamma, alpha, iterations, evaluate_every, target_update_freq, model_save_path
    )
    print(f"Ambiente: {env_name}")
    print(f"Recompensa média final: {np.mean(avg_rewards)}")
    print(f"Variância da recompensa: {np.var(avg_rewards)}")
    print("-" * 50)

In [63]:
train_cliffwalking()

Run sarsa in <OrderEnforcing<PassiveEnvChecker<CliffWalkingEnv<CliffWalking-v0>>>>
Episódio 0 terminou com recompensa -213 na transição 113
Episódio 100 terminou com recompensa -1492 na transição 50534
Episódio 200 terminou com recompensa -1271 na transição 86760
Episódio 300 terminou com recompensa -130 na transição 115744
Episódio 400 terminou com recompensa -84 na transição 140906
Episódio 500 terminou com recompensa -143 na transição 165407
Episódio 600 terminou com recompensa -399 na transição 187304
Episódio 700 terminou com recompensa -547 na transição 207077
Episódio 800 terminou com recompensa -122 na transição 225002
Episódio 900 terminou com recompensa -392 na transição 240924
Episódio 1000 terminou com recompensa -296 na transição 257766
Episódio 1100 terminou com recompensa -74 na transição 272250
Episódio 1200 terminou com recompensa -65 na transição 287210
Episódio 1300 terminou com recompensa -45 na transição 300215
Episódio 1400 terminou com recompensa -177 na transiçã